https://softwareengineering.stackexchange.com/questions/273302/what-is-the-relationship-between-scope-and-namespaces-in-python
https://docs.python.org/3/reference/simple_stmts.html#the-global-statement
https://data-flair.training/blogs/python-namespace-and-variable-scope/
https://data-flair.training/blogs/python-variable-scope/#

Global scope = outermost enclosing scope ?

built-in locals() : retourne un dictionnaire représeantant le namespace local. Inclut les variables libres si appelé de l'intérieur d'une fonction.
built-in globals() : retourne un dictionnaire représentant le namespace global courant. Il s'agit toujours du namespace du module courant, si appelé de l'intérieur d'une fonction ou d'une méthode, il s'agira de celui du module où la fonction/méthode est définie.

https://stackoverflow.com/questions/1024049/is-it-pythonic-to-import-inside-functions

Fonction dir() sans arguments retourne la liste des noms actuellement présents dans le scope local (globalement retourne la une liste correspondant aux clés du dictionnaire retourné par locals()) Ce qui est retourné dépend de l'objet qui lui est passé : https://docs.python.org/3/library/functions.html#dir

On y trouve notamment __name__, __package__.

In [9]:
from collections import OrderedDict
import pandas as pd

In [19]:
b = 2
def someFunc():
    from collections import OrderedDict as OD
    x = 1
    print(b)
    print(locals())
    
someFunc()

2
{'OD': <class 'collections.OrderedDict'>, 'x': 1}


The import statement combines two operations; it searches for the named module, then it binds the results of that search to a name in the local scope.

import pandas +> le nom pandas est ajouté au namespace courant et lié à l'objet module correspondant 
import pandas as pd +> même chose sauf que le nom auquel est lié l'objet module correspondant à pandas est pd.
from collections import OrderedDict +> On ajoute le nom OrderedDict au namespace courant qui est lié à (l'objet) la classe collections.OrderedDict. On peut appeler l'objet directement par son nom.

Les objets du module importé sont disponibles comme attributs de l'objet module. Exemple : la fonction someFunc du module foo importé via import foo sera disponible comme foo.someFunc. La table des symboles du module est attachée à l'attribut __dict__ dont la built-in function dir retourne les noms (clés du dictionnaire __dict__). 

In [34]:
len(dir())

81

## LEGB rule
Python implémente le scoping lexical. Il existe 4 types de scope en Python, ces derniers étant organisés suivant une hiérarchie. Pour résoudre les noms, Pyhton parcourt ces scopes séquentiellement, du plus étroit au plus large (L > E > G > B) : 
* Local : scope le plus étroit, il correspond à l'ensemble des noms définis dans bloc de code local (fonction, classe).
* Enclosing : ensemble des noms définis au niveau où le code exécuté a été déclaré. 
* Global : ensemble des noms définis au niveau du module courant. On peut le voir comme le outermost enclosing scope.
* Built-in : ensemble des noms nativement intégérés au langage.

Python fournit deux built-in functions permettant de lister l'ensemble des noms définis dans le scope courant (```locals()```) et global (```globals()```).

Pyhton fournit également deux built-in statements permettant d'outrepasser la LEGB rule localement, à l'intérieur d'un bloc de code. L'effet de ces déclaration portent sur l'ensemble du bloc : 
* ```global``` : tous les identifiants suivant cette déclaration sont à résoudre dans le scope global 
* ```nonlocal``` : tous les identifiants suivant cette déclaration sont à résoudre dans l'enclosing scope le plus proche, le scope global étant exclu.

Ces déclarations permettent notamment d'agir sur des variables qui ne sont pas dans notre scope. Une telle utilisation n'est pas recommandée en raison de la moindre prédictibilité du comportement du code. Si on souhaite modfier une variable d'un certain scope par une fonction, la concevoir de façon à ce qu'on lui passe la valeur et qu'elle la retourne éventuellemet, c'est plus explicite.

In [10]:
print("--------Reference--------")
# Global 
x = 0
def outer():
    # Enclosing
    x = 1
    def inner():
        # Local
        x = 2
        print("x in local scope:", x)

    inner()
    print("x in enclo. scope:", x)

outer()
print("x in global scope:", x)

print("--------global--------")
# Global 
x = 0
def outer():
    # Enclosing
    x = 1
    def inner():
        # Local
        global x
        x = 2
        print("x in local scope:", x, "(actually value in global scope)")

    inner()
    print("x in enclo. scope:", x)

outer()
print("x in global scope:", x)

print("--------nonlocal--------")
# Global 
x = 0
def outer():
    # Enclosing
    x = 1
    def inner():
        # Local
        nonlocal x
        x = 2
        print("x in local scope:", x,  "(actually value in enclosing scope)")

    inner()
    print("x in enclo. scope:", x)

outer()
print("x in global scope:", x)

--------Reference--------
x in local scope: 2
x in enclo. scope: 1
x in global scope: 0
--------global--------
x in local scope: 2 (actually value in global scope)
x in enclo. scope: 1
x in global scope: 2
--------nonlocal--------
x in local scope: 2 (actually value in enclosing scope)
x in enclo. scope: 2
x in global scope: 0


In [ ]:
def make_counter():
    count = 0
    def counter():
        nonlocal count
        count += 1
        return count
    return counter

# Design auquel on peut préférer un générateur 

def counter_generator():
    count = 0
    while True:
        count += 1
        yield count

## Mutable default value
Utiliser une structure mutable comme valeur par défaut d'un ou plusieurs arguments d'une fonction est dangereux et considéré comme un anti-pattern. Python va en effet lors du premier appel à la fonction créer un objet pour la valeur par défaut et qui c'est le même objet qui sera utilisé pour les appels ultérieurs à la fonction. Que cet objet soit mutable, c'est prendre le risque que notre valeur par défaut ne soit pas la même pour tous les appels à la fonction.

In [1]:
def func(x, y=[]):
    y.append(x)
    print(y)
    
func(1)
func(10)
func(100)

[1]
[1, 10]
[1, 10, 100]


Préférer cette forme : 

In [3]:
def recommendedFunc(x, y=None):
    if not y:
        y = []
    y.append(x)
    print(y)
    
recommendedFunc(1)
recommendedFunc(10)
recommendedFunc(100)

[1]
[10]
[100]


### Function argument checking en Python 
Finalement checker le type c'est tenter d'implémenter du typage statique? 
Type-checking should be avoided as much as possible as it goes against duck-typing (isinstance ?).

The only place where validation really makes sense is at system or subsystem entry point, such as web forms, command line arguments, etc. Everywhere else, as long as your functions are properly documented, it's the caller's responsibility to pass appropriate arguments.

https://stackoverflow.com/questions/19684434/best-way-to-check-function-arguments-in-python/19684962#19684962
https://medium.com/@ageitgey/learn-how-to-use-static-type-checking-in-python-3-6-in-10-minutes-12c86d72677b

EAFP
Easier to ask for forgiveness than permission. This common Python coding style assumes the existence of valid keys or attributes and catches exceptions if the assumption proves false. This clean and fast style is characterized by the presence of many try and except statements. The technique contrasts with the LBYL style common to many other languages such as C.

LBYL

Look before you leap. This coding style explicitly tests for pre-conditions before making calls or lookups. This style contrasts with the EAFP approach and is characterized by the presence of many if statements.

https://stackoverflow.com/questions/12265451/ask-forgiveness-not-permission-explain
http://stupidpythonideas.blogspot.com/2015/05/if-you-dont-like-exceptions-you-dont.html
Many people—especially people coming from Java—think that using try/except is "inelegant", or "inefficient". Or, slightly less meaninglessly, they think that "exceptions should only be for errors, not for normal flow control".

https://dbader.org/blog/python-assert-tutorial
https://jeffknupp.com/blog/2013/02/06/write-cleaner-python-use-exceptions/

The principle of duck typing says that you shouldn't care what type of object you have - just whether or not you can do the required action with your object. For this reason the isinstance keyword is frowned upon.

https://softwareengineering.stackexchange.com/a/287940 premier paragraphe : isinstance est forcément trop restrictif et en cela contraire à l'esprit du duck typing. Ne parlons même pas de astuces à la hasattr qui permettent certes de capturer plus de types mais assez maladroitement. Au fond aller à l'encontre de l'esprit du duck typing / du langage n'est qu'un choix politique ? Ca ne marche pas beaucoup moins bien, c'est juste moins pythonic 
https://softwareengineering.stackexchange.com/a/287934 : Si t'as passé le mauvais truc, c'est ta faute.
Le duck typing est cohérent avec le style de programmation en EAFP : on fait comme si l'objet passé est le bon et si ce n'est pas le cas c'est de notre faute et ça lancera une exception qu'on doit gérer (d'où l'utilisation de blocs try/except dans les endroit où on peut se prendre des erreurs liées au type de ce que l'on passe.

ABC vs duck-typing 
https://stackoverflow.com/questions/18430970/abstract-base-classes-versus-duck-typing
https://stackoverflow.com/a/4549229 Avant dernier paragraphe
https://www.python.org/dev/peps/pep-3119/#abcs-vs-alternatives 
http://docs.python.org/glossary.html => duck-typing

Assert
https://dbader.org/blog/python-assert-tutorial
https://stackoverflow.com/questions/46723564/when-should-i-use-assert-in-python
When to use assert : http://archive.fo/5GfiG 
assert n'est pas fait pour faire du type/argument checking. C'est principalement une aide au débugage et permet de s'assurer en un point du programme que tout s'est passé comme attendu avant (ce qui contribue au débugage).
They shouldn't be used to control program flow.

Asserts should be used to test conditions that should never happen. The purpose is to crash early in the case of a corrupt program state.

Exceptions should be used for errors that can conceivably happen, and you should almost always create your own Exception classes.

Globalement préférer une syntaxe EAFP. Si en revanche on sait qu'une situation peut se produire et qui mène inévitablement à une erreur => Programmation défensive / LBYL ? if + raise ? (if + raise vs assert => assert n'est pas là pour faire de la data validation). EAFP avec try/except/finally/else est plus pythonic mais le choix entre un style LBYL et EAFP peut parfois plus être une question politique (?).

Sur les pitfalls de LBYL : https://docstore.mik.ua/orelly/other/Docs/oreilly/other2/python/0596001886_pythonian-chp-6-sect-6.html

In [7]:
import abc

class TestABC(abc.ABC):
    
    @abc.abstractmethod
    def method_one(self):
        pass
    
    @abc.abstractmethod
    def method_two(self):
        pass
    
class TestConcreteClass():
    
    def method_one(self):
        print('LOL')
        
    def method_two(self):
        print('fck')
        
a = TestConcreteClass()

print(issubclass(TestConcreteClass, TestABC))
print(isinstance(a, TestABC))

a.method_three()

False
False


AttributeError: 'TestConcreteClass' object has no attribute 'method_three'

The collections.abc submodule has some ABCs that can be used to test whether a class or instance provides a particular interface
http://sametmax.com/quest-ce-que-le-duck-typing-et-a-quoi-ca-sert/
A quoi ça sert de faire un hasattr ? vs un try / except AttributeError qui est plus conforme ? Le AttributeError ne capte que les cas où la méthode/l'attribut qu'on cherche n'est pas implémenté, à compléter avec d'éventuelles exceptions complémentaires pour les cas où la méthode foire. 

if hasattr(x, someattr):
    try:
        x.someattr()
    except Exception:
        pass
        
moins élégant que 

try:
    x.someattr()
except AttributeError:
    pass
except Exception:
    pass
    
Protocole en Python : s'entend comme une interface informelle, uniquement définie dans la documentation (exemple : protocole iterator) et qu'on ne peut pas tester comme tel. On peut implémenter un protocole en implémentant les méthodes qui le définissent (ex: __iter__) grace au duck-typing : il suffit d'implémenter ce qu'il faut pour que ça marche. Le duck-typing n'impose pas la définition d'interfaces formelles, on peut donc se contenter de protocoles.

Any objects deriving from these (abstract) bases classes are forced to implement those methods. If we see an object has our class as a base class, we can say that this object implements the interface. That is now we can use types to tell if an object implements a certain interface.

Les protocoles peuvent bien marcher et ne pas poser de problèmes dans la majorité des cas mais il existe des cas où les interfaces informelles et le duck typing peuvent créer de la confusion et c'est là que les ABC viennent compléter le duck typing en permettant la définition d'interfaces formelles dont on contrôle l'implémentation à l'aide du type (isinstance/issubclass).

Remarque : pour qu'un objet implémente l'interface définie par une ABC, il faut soit que sa classe en hérite soit que sa classe soit enregistrée comme sous-classe virtuelle (virtual subclass) de l'ABC. Plus généralement, la relation d'appartenance est définie par les implémentations des méthodes magiques (définies au niveau de la métaclasse) __checkinstance__ et __checksubclass__ respectivement appelée lors du recours aux built-ins isinstance (appelée sur l'objet) et issubclass (appelée sur la classe). L'héritage est au fond la meilleure assurance de l'implémentation de l'interface, la classe ne pouvant être instanciée si elle n'implémente pas toutes les méthodes abstraites de l'ABC.

Exemple de cas où les ABC viennent affiner une distinction impossible avec le duck-typing seul :

In [10]:
import abc

class Bird(abc.ABC):
    @abc.abstractmethod
    def fly(self):
        pass
    
class Plane(abc.ABC):
    @abc.abstractmethod
    def fly(self):
        pass

class Parrot(Bird):
    
    def fly(self):
        print("Flies")
        
class Airbus(Plane):
    
    def fly(self):
        print("Flies")
        
kiki = Parrot()
a380 = Airbus()

print(isinstance(kiki, Airbus))
print(isinstance(a380, Bird))

# Les deux possèdent une méthode fly mais on peut désormais faire la différence entre l'objet qui implémente l'interface
# Plane de celui qui implémente l'interface Bird.

False
False


In [ ]:
# Le module abc va plus loin, ils vous permet de définir une classe mère abstraite à des classes déjà existantes 
# grâce à la méthode de classe register.

@Plane.register
class Boeing:
    pass

b777 = Boeing()

isinstance(b777, Plane)

# L'objet de la classe Boeing est reconnu et accepté comme implémentant l'interface Plane (même si ça peut ne pas être 
# le cas).
# On peut éventuellement surcharger isinstance/issubclass pour en augmenter le comportement

Pourquoi faire hériter explicitement ses classes de object ?

Comment surcharger __checksubclass__ sans avoir à réimplémenter entièrement celle de ABCMeta ? => on implémente la méthode __subclasshook__ dans la classe abstraite (voir comment bien le faire dans le PEP dédié, https://stackoverflow.com/questions/9223760/how-do-you-verify-duck-typed-interfaces-in-python#).

Remarque en faire une classmethod : ça lui permet d'être appelée par toutes les sousclasses de la base classe qui hérite de la classe abstraite (?). 

On peut ainsi grace à __subclasshook__ dire qu'une classe implémente l'inferface si elle implémente une liste de méthodes définies dans __subclasshook__

In [13]:
import abc

class TestABC(abc.ABC):
    
    @abc.abstractmethod
    def method_one(self):
        pass
    
    @abc.abstractmethod
    def method_two(self):
        pass
    
    # Une implémentation possible, copiée de SO
    @classmethod
    def __subclasshook__(cls, C):
        required = ["method_one", "method_two"]
        rtn = True
        for r in required:
            if not any(r in B.__dict__ for B in C.__mro__):
                rtn = NotImplemented
        return rtn
    
class TestConcreteClass():
    
    def method_one(self):
        print('LOL')
        
    def method_two(self):
        print('fck')
        
a = TestConcreteClass()

print(issubclass(TestConcreteClass, TestABC))

# TestConcreteClass est ici reconnue comme implémentant l'interface alors qu'elle n'hérite ni de la classe abstraite
# ni d'est enregistrée comme sous classe virtuelle de celle-ci. On a juste augmenté grace à __subclasshook__ la notion
# d'implémentation de l'interface.

True


L'exemple ci-dessus peut déplaire (on pourrait à la limite se demander de l'utilité de ABCMeta qu'on vient de court-circuiter). Implémenter une interface ce n'est pas simplement implémenter un certain nombre de méthodes c'est une compréhension partagée de ce que chaque méthode doit faire : en héritant de la classe abstraite et en s'étant astreint à en implémenter toutes les méthodes abstraites, il est sous-entendu que le client a bien lu et compris le contrat, les commentaires et descriptions de ce qu'on attend des méthodes abstraites rédigés par le concepteur de l'interface. En héritant de l'interface, on promet d'en suivre toutes les règles (et pas seulement de remplir de code le corps des membres abstraits).

Rationale behind using ABCs (vs duck typing) : 

Python's duck-typing has many advantages in flexibility over static-typing, but it doesn't solve all the problems. ABCs offer an intermediate solution between the free-form of Python and the bondage-and-discipline of a staticly-typed language. 

That said, I find that I rarely need to write my own ABC and I typically discover the need for one through refactoring. If I see a polymorphic function making a lot of attribute checks, or lots of functions making the same attribute checks, that smell suggests the existence of an ABC waiting to be extracted. https://stackoverflow.com/a/19328146

In the context of Object Oriented Programming, a protocol is an informal interface, defined only in documentation and not in code. For example, the sequence protocol in Python entails just the __len__ and __getitem__ methods.  ABCs make protocols explicits.

Python c'est du duck typing le type n'importe pas du moment que la méthode demandée existe mais alors, a-t-on besoin de classe abstraite ? Je vois deux raisons à leur utilisation... La première, centraliser la doc et avoir une interface (ensemble des signatures des méthodes d'un type) explicite. La deuxième raison est que si un objet a beaucoup de méthodes à redéfinir, c'est plus facile de faire un seul isinstance à l'entrer d'une fonction que de un hasattr par méthode (ce qui peut engendrer du code assez confus).

What I talk about are of course formal interfaces. I see your grimace on your face. "What is he talking about? We have duck-typing" you'll say. Yes, with all my heart I am for duck-typing! But at some complexity level duck-typing is not enough.

Avantage de la classe abstraite : si la méthode n'est pas implémentée, on ne peut pas instancier l'objet. C'est toujours mieux que de lancer de long traitements dessus et qu'au bout d'un certain temps tout crash à cause de la méthode manquante dont l'absence se révèle à l'exécution.

http://kartowicz.com/dryobates/2015-05/interfaces/ : semble pas mal revenir sur l'argument nuémro 2

ABCs / formal interfaces : pas obligatoires mais il y a des moments où le duck typing devient lourd et maladroit et c'est là qu'elles peuvent ramener de la clarté dans le code.